NOTEBOOK CREATED BY SHREYAS KAWALE IN 'CLUS' DATABRICKS CLUSTER- CLUSTER HELPS IN RESOURCE MANAGEMENT AND COST OPTIMIZATION

In [0]:
# File location and type
file_location = "/FileStore/tables/car_sales.csv"
file_type = "csv"
df_car= spark.read.csv(file_location, header=True, inferSchema=True)


In [0]:
df_car.head(7)

Out[10]: [Row(Make='Toyota', Colour='White', Odometer (KM)=150043, Doors=4, Price='$4,000.00'),
 Row(Make='Honda', Colour='Red', Odometer (KM)=87899, Doors=4, Price='$5,000.00'),
 Row(Make='Toyota', Colour='Blue', Odometer (KM)=32549, Doors=3, Price='$7,000.00'),
 Row(Make='BMW', Colour='Black', Odometer (KM)=11179, Doors=5, Price='$22,000.00'),
 Row(Make='Nissan', Colour='White', Odometer (KM)=213095, Doors=4, Price='$3,500.00'),
 Row(Make='Toyota', Colour='Green', Odometer (KM)=99213, Doors=4, Price='$4,500.00'),
 Row(Make='Honda', Colour='Blue', Odometer (KM)=45698, Doors=4, Price='$7,500.00')]

In [0]:
#[Odometer(Km),Doors]----> new feature--->independent feature
#SAMPLE PREDICTION WITH THESE TWO COLUMNS

In [0]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=["Odometer (KM)","Doors"],outputCol="Independent Features")

In [0]:
output=featureassembler.transform(df_car)

In [0]:
output.show()

+------+------+-------------+-----+----------+--------------------+
|  Make|Colour|Odometer (KM)|Doors|     Price|Independent Features|
+------+------+-------------+-----+----------+--------------------+
|Toyota| White|       150043|    4| $4,000.00|      [150043.0,4.0]|
| Honda|   Red|        87899|    4| $5,000.00|       [87899.0,4.0]|
|Toyota|  Blue|        32549|    3| $7,000.00|       [32549.0,3.0]|
|   BMW| Black|        11179|    5|$22,000.00|       [11179.0,5.0]|
|Nissan| White|       213095|    4| $3,500.00|      [213095.0,4.0]|
|Toyota| Green|        99213|    4| $4,500.00|       [99213.0,4.0]|
| Honda|  Blue|        45698|    4| $7,500.00|       [45698.0,4.0]|
| Honda|  Blue|        54738|    4| $7,000.00|       [54738.0,4.0]|
|Toyota| White|        60000|    4| $6,250.00|       [60000.0,4.0]|
|Nissan| White|        31600|    4| $9,700.00|       [31600.0,4.0]|
+------+------+-------------+-----+----------+--------------------+



In [0]:
finalized_data=output.select("Independent Features","Price")
finalized_data.show()

+--------------------+----------+
|Independent Features|     Price|
+--------------------+----------+
|      [150043.0,4.0]| $4,000.00|
|       [87899.0,4.0]| $5,000.00|
|       [32549.0,3.0]| $7,000.00|
|       [11179.0,5.0]|$22,000.00|
|      [213095.0,4.0]| $3,500.00|
|       [99213.0,4.0]| $4,500.00|
|       [45698.0,4.0]| $7,500.00|
|       [54738.0,4.0]| $7,000.00|
|       [60000.0,4.0]| $6,250.00|
|       [31600.0,4.0]| $9,700.00|
+--------------------+----------+



In [0]:
from pyspark.ml.regression import LinearRegression
##train test split
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='Independent Features', labelCol='Price')
regressor=regressor.fit(train_data)

---------------------------------------------------------------------------
IllegalArgumentException                  Traceback (most recent call last)
File <command-617528596635789>:5
      3 train_data,test_data=finalized_data.randomSplit([0.75,0.25])
      4 regressor=LinearRegression(featuresCol='Independent Features', labelCol='Price')
----> 5 regressor=regressor.fit(train_data)

File /databricks/python_shell/dbruntime/MLWorkloadsInstrumentation/_pyspark.py:30, in _create_patch_function.<locals>.patched_method(self, *args, **kwargs)
     28 call_succeeded = False
     29 try:
---> 30     result = original_method(self, *args, **kwargs)
     31     call_succeeded = True
     32     return result

File /databricks/spark/python/pyspark/ml/base.py:205, in Estimator.fit(self, dataset, params)
    203         return self.copy(params)._fit(dataset)
    204     else:
--> 205         return self._fit(dataset)
    206 else:
    207     raise TypeError(
    208         "Params must be either 

In [0]:
from pyspark.sql.functions import col, regexp_replace
from pyspark.sql.types import IntegerType, FloatType

# Assuming you have a DataFrame named "df" with a "price" column containing values like "$4,000.00"

# Remove the currency symbol ($) and comma (,) from the "price" column
df_car = df_car.withColumn("price", regexp_replace(col("price"), "[$,]", ""))

In [0]:
df_car.show()

+------+------+-------------+-----+--------+
|  Make|Colour|Odometer (KM)|Doors|   price|
+------+------+-------------+-----+--------+
|Toyota| White|       150043|    4| 4000.00|
| Honda|   Red|        87899|    4| 5000.00|
|Toyota|  Blue|        32549|    3| 7000.00|
|   BMW| Black|        11179|    5|22000.00|
|Nissan| White|       213095|    4| 3500.00|
|Toyota| Green|        99213|    4| 4500.00|
| Honda|  Blue|        45698|    4| 7500.00|
| Honda|  Blue|        54738|    4| 7000.00|
|Toyota| White|        60000|    4| 6250.00|
|Nissan| White|        31600|    4| 9700.00|
+------+------+-------------+-----+--------+



In [0]:
df_car = df_car.withColumn("Price", col("Price").cast(FloatType()))

# Convert the "price" column from float to integer
df_car = df_car.withColumn("Price", col("Price").cast(IntegerType()))

In [0]:
df_car.printSchema()

root
 |-- Make: string (nullable = true)
 |-- Colour: string (nullable = true)
 |-- Odometer (KM): integer (nullable = true)
 |-- Doors: integer (nullable = true)
 |-- Price: integer (nullable = true)



In [0]:
# CODINT THE SAME MACHINE LEARNING
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=["Odometer (KM)","Doors"],outputCol="Independent Features")
output=featureassembler.transform(df_car)
finalized_data=output.select("Independent Features","Price")
from pyspark.ml.regression import LinearRegression
##train test split
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='Independent Features', labelCol='Price')
regressor=regressor.fit(train_data)

In [0]:
pred_results=regressor.evaluate(test_data)
pred_results.predictions.show()

+--------------------+-----+------------------+
|Independent Features|Price|        prediction|
+--------------------+-----+------------------+
|       [32549.0,3.0]| 7000|-3871.531555745576|
|       [45698.0,4.0]| 7500| 8001.782638669749|
|       [60000.0,4.0]| 6250| 7364.240365169411|
|       [99213.0,4.0]| 4500| 5616.236927134953|
|      [213095.0,4.0]| 3500|  539.702932535205|
+--------------------+-----+------------------+



In [0]:
pred_results.meanAbsoluteError,pred_results.meanSquaredError

Out[53]: (3312.817710836897, 25938571.876111083)